In [10]:
import sys
from pysc2.lib import app
from argparse import Namespace

# Parse the flags so that the default values are ready.
app.FLAGS([sys.argv[0]])

from pysc2 import maps
from pysc2.env import available_actions_printer
from pysc2.env import run_loop
from pysc2.env import sc2_env
from pysc2.agents.base_agent import *
from pysc2.agents.random_agent import *
from pysc2.agents.scripted_agent import *
from pysc2.lib import actions
import time
import threading
from Queue import Queue

import numpy
import numpy as np

import matplotlib.pyplot as pp

In [11]:
%matplotlib inline

In [2]:
env = sc2_env.SC2Env(map_name="CollectMineralShards", step_mul=1, visualize=True)

In [3]:
total_frames = 0
start_time = time.time()
action_spec = env.action_spec()
observation_spec = env.observation_spec()

In [4]:
# This resets a game.
timesteps = env.reset()

In [20]:
_PLAYER_RELATIVE = features.SCREEN_FEATURES.player_relative.index
_PLAYER_FRIENDLY = 1
_PLAYER_NEUTRAL = 3  # beacon/minerals
_NO_OP = actions.FUNCTIONS.no_op.id
_MOVE_SCREEN = actions.FUNCTIONS.Move_screen.id
_SELECT_ARMY = actions.FUNCTIONS.select_army.id
_SELECT_POINT = actions.FUNCTIONS.select_point.id
_NOT_QUEUED = [0]
_SELECT_ALL = [0]

In [26]:
# Collect Mineral v2, two units go to closest for them separately.
last_player = None
for t in range(1000):
  obs = timesteps[0]

  if _MOVE_SCREEN in obs.observation["available_actions"]:
    player_relative = obs.observation["screen"][_PLAYER_RELATIVE]
    neutral_y, neutral_x = (player_relative == _PLAYER_NEUTRAL).nonzero()
    player_y, player_x = (player_relative == _PLAYER_FRIENDLY).nonzero()
    if not neutral_y.any() or not player_y.any():
      acts = actions.FunctionCall(_NO_OP, [])
    else:
      i = np.random.randint(len(player_x))
      player = [player_x[i], player_y[i]]
      if t % 2 == 0:
        acts = actions.FunctionCall(_SELECT_POINT, [[0], player])
        last_player = player
      else:
        closest, min_dist = None, None
        for p in zip(neutral_x, neutral_y):
          dist = numpy.linalg.norm(numpy.array(last_player) - numpy.array(p))
          if not min_dist or dist < min_dist:
            closest, min_dist = p, dist
        acts = actions.FunctionCall(_MOVE_SCREEN, [_NOT_QUEUED, closest])
  else:
    acts = actions.FunctionCall(_SELECT_ARMY, [_SELECT_ALL])

  timesteps = env.step([acts])

In [6]:
# Collect Mineral
for t in range(10):
  obs = timesteps[0]

  if _MOVE_SCREEN in obs.observation["available_actions"]:
    player_relative = obs.observation["screen"][_PLAYER_RELATIVE]
    neutral_y, neutral_x = (player_relative == _PLAYER_NEUTRAL).nonzero()
    player_y, player_x = (player_relative == _PLAYER_FRIENDLY).nonzero()
    if not neutral_y.any() or not player_y.any():
      acts = actions.FunctionCall(_NO_OP, [])
    else:
      player = [int(player_x.mean()), int(player_y.mean())]
      closest, min_dist = None, None
      for p in zip(neutral_x, neutral_y):
        dist = numpy.linalg.norm(numpy.array(player) - numpy.array(p))
        if not min_dist or dist < min_dist:
          closest, min_dist = p, dist
      acts = actions.FunctionCall(_MOVE_SCREEN, [_NOT_QUEUED, closest])
  else:
    acts = actions.FunctionCall(_SELECT_ARMY, [_SELECT_ALL])

  timesteps = env.step([acts])

In [5]:
# Random Actions
for t in range(100):
  obs = timesteps[0]
  function_id = numpy.random.choice(obs.observation["available_actions"])
  args = [[numpy.random.randint(0, size) for size in arg.sizes]
          for arg in action_spec.functions[function_id].args]
  print "%s (%d)" % (actions.FUNCTIONS[function_id].name, function_id),
  for i in range(len(args)):
    print action_spec.functions[function_id].args[i].sizes, args[i],
  print
  acts = actions.FunctionCall(function_id, args)
  timesteps = env.step([acts])

select_rect (3) (2,) [0] (64, 64) [25, 59] (64, 64) [5, 4]
select_army (7) (2,) [1]
select_army (7) (2,) [1]
select_control_group (4) (5,) [0] (10,) [2]
select_point (2) (4,) [1] (64, 64) [53, 14]
Stop_quick (453) (2,) [1]
Attack_screen (12) (2,) [0] (64, 64) [20, 56]
no_op (0)
Smart_minimap (452) (2,) [1] (64, 64) [26, 43]
Move_screen (331) (2,) [1] (64, 64) [32, 7]
Smart_minimap (452) (2,) [0] (64, 64) [60, 62]
select_rect (3) (2,) [1] (64, 64) [17, 21] (64, 64) [55, 44]
select_unit (5) (4,) [3] (500,) [29]
Patrol_screen (333) (2,) [0] (64, 64) [2, 32]
Stop_quick (453) (2,) [0]
Patrol_minimap (334) (2,) [0] (64, 64) [36, 56]
select_army (7) (2,) [1]
select_unit (5) (4,) [0] (500,) [217]
Patrol_minimap (334) (2,) [0] (64, 64) [19, 22]
Attack_screen (12) (2,) [0] (64, 64) [18, 48]
select_point (2) (4,) [0] (64, 64) [13, 21]
Patrol_screen (333) (2,) [0] (64, 64) [32, 30]
Attack_minimap (13) (2,) [0] (64, 64) [28, 7]
move_camera (1) (64, 64) [36, 1]
HoldPosition_quick (274) (2,) [1]
Stop

In [27]:
env.close()